<a href="https://colab.research.google.com/github/RK22000/FOLd/blob/main/FOLd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

This document is divided into 3 sections:

1. Helper Methods

        Has all the helper methods and their minor tests. (Skipable)
1. Main Code
    
        Has the code for the main brains of the First Order Logic.
1. Tests
    
        Shows the FOL engine at work with test cases. Can be checked before reading other sections.



# Helper methods

In [1]:
import re

In [2]:
re.findall(r'\(.*\)', "Smart(x)(y)")

['(x)(y)']

In [3]:
re.findall(r'\([^(]*\)', "Smart(x)(y)")

['(x)', '(y)']

In [4]:
#match = re.findall(r'\((?P<arguments>[^()]*)\)', "Smart(x, y) Clever(p, (q))")
b1 = r'\(([^)(]*)\)'
b2 = r'\(([^)(]*' + r'\([^)(]\)' + r'[^()]*)\)'

match = re.findall(b1+'|'+b2, "Smart(x, y) Clever(p, (q))")
match#.group("arguments")

[('x, y', ''), ('', 'p, (q)')]

In [5]:
import re
def getFuncArgs(predicate: str):
    """Get the first function and its set of arguments in the predicate"""
    regex = re.compile(r'(?P<negation>~?)(?P<function>\w+)\((?P<arguments>[^()]*)\)')
    match = regex.search(predicate)
    arguments = match.group("arguments").split(',')
    for i, argument in enumerate(arguments):
        arguments[i] = argument.strip()
    func = match.group("function")
    negation = match.group("negation")
    return func, arguments, negation


In [6]:
getFuncArgs("~Smart(x, y)")

('Smart', ['x', 'y'], '~')

In [21]:
import re
def isFuncArgs(predicate: str):
    """Get the first function and its set of arguments in the predicate"""
    regex = re.compile(r'^(?P<function>~?\w+)\((?P<arguments>[^()]*)\)$')
    match = regex.search(predicate)
    return True if match else False


In [67]:
isFuncArgs("Smart(x)^Clever(x)")

False

In [22]:
isFuncArgs("abc")

False

In [9]:
def ValidTerm(term: str):
    regex = re.compile(r"^[A-Za-z]+$")
    return True if regex.search(term) else False

def GroundTerm(term: str):
    if not ValidTerm(term=term): return False
    regex = re.compile(r"^[a-z]+$")
    return True if regex.search(term) else False

def ConstantTerm(term: str):
    if not ValidTerm(term=term): return False
    regex = re.compile(r"^[A-Z][A-Za-z]+$")
    return True if regex.search(term) else False

In [10]:
GroundTerm('x'), ConstantTerm('RahulKan'), ValidTerm('XyZ')

(True, True, True)

In [11]:
def swapable(f_terms, p_terms):
    count=0
    if not len(f_terms) == len(p_terms): return False
    for f_term, p_term in zip(f_terms, p_terms):
        if GroundTerm(f_term) or f_term==p_term:
            count+=1
    return count==len(f_terms)

In [12]:
None == True, None == False

(False, False)

In [13]:
def isRule(sentence: str):
    return sentence.count("=>") == 1

In [14]:
isRule(" a => b")

True

In [15]:
def getCondImpl(sentence: str):
    condition, implication = sentence.split("=>")
    return condition.strip(), implication.strip()

In [16]:
getCondImpl(" a ^ x=> b ")

('a ^ x', 'b')

In [17]:
def swap_list(f_terms: list, p_terms: list):
    swap = dict()
    if not len(f_terms) == len(p_terms): return swap
    for f_term, p_term in zip(f_terms, p_terms):
        if GroundTerm(f_term):
            swap[f_term] = p_term
    return swap

In [111]:
def replace_var(var: str, rpl: str, sentence:str) -> str:
    def swap(match: re.Match):
        #print(f'Match {match}')
        return match.group()[0] + rpl + match.group()[-1]
    wrapper = r"[\s)(,]"
    pattern = wrapper + f"(?P<var>{var})" + wrapper
    return re.sub(pattern=pattern, repl=swap, string=sentence)

In [108]:
replace_var("y", "Azula", "Prodigy(y)")

Match <re.Match object; span=(7, 10), match='(y)'>


'Prodigy(Azula)'

In [54]:
def conjunction(sentence: str) -> bool:
    conjuncts = set( [conj.strip() for conj in sentence.split("^")] )
    return "^" in sentence and '' not in conjuncts

In [62]:
conjunction(" a ^ B  ^ ^ x")

False

In [63]:
def getConjuncts(sentence: str) -> set:
    return set([conj.strip() for conj in sentence.split("^")])


In [66]:
getConjuncts(" a ^ B ^ x ")

{'B', 'a', 'x'}

# Main Code

In [112]:
class Fol_engine:
    def __init__(self, facts: set = set(), rules: set = set()) -> None:
        self.facts = facts
        self.rules = rules


    def isTrue(self, predicate: str):

        if conjunction(sentence=predicate):
            conjuncts = getConjuncts(sentence=predicate)
            print(f"Conjunct of {conjuncts}")
            result = True
            for conjunct in conjuncts:
                val = self.isTrue(conjunct)
                if val == False: # val may be False or None
                    return False
                elif val is None:
                    result = None
            return result
        
        elif isFuncArgs(predicate):
            p_func, p_arguments, p_negation = getFuncArgs(predicate)

            print(f"Checking {predicate}")

            for fact in self.facts:
                if isFuncArgs(fact):
                    f_func, f_arguments, f_negation = getFuncArgs(fact)
                    if f_func==p_func and len(p_arguments)==len(f_arguments):
                        print(f'Fact Function match \'{p_func}\'')
                        print(f'p_arguments {p_arguments}, f_arguments {f_arguments}')

                        if swapable(f_terms=f_arguments, p_terms=p_arguments):
                            mod = "" if p_negation == f_negation else "not "
                            print(f'Fact \'{fact}\' |=> {mod}{predicate}')
                            return p_negation == f_negation
                        else:
                            print(f'Fact \'{fact}\' ?=> {predicate}')

            for rule in self.rules:
                if isRule(rule):
                    condition, implication = getCondImpl(rule)
                    if isFuncArgs(implication):
                        i_func, i_arguments, i_negation = getFuncArgs(implication)
                        if i_func==p_func and len(i_arguments)==len(p_arguments):
                            print(f'Rule match \'{rule}\' for \'{predicate}\'')

                            if swapable(f_terms=i_arguments, p_terms=p_arguments):
                                swap = swap_list(f_terms=i_arguments, p_terms=p_arguments)
                                print(f'Swaping {swap}')

                                for var in swap:
                                    condition = replace_var(var=var, rpl=swap[var], sentence=condition)
                                    rule = replace_var(var=var, rpl=swap[var], sentence=rule)
                                    #condition = re.sub(var, swap[var], condition)
                                    #rule = re.sub(var, swap[var], rule)
                                
                                print(f'Swapped condition \'{condition}\'')
                                result = self.isTrue(condition)
                                if result == True:
                                    print(f'Rule {rule}')
                                    return result if p_negation == i_negation else not result
        return None

# Tests

## Test 7

In [113]:
print("Test 7.1")
def test():
    facts = {
        "TeamAvatarMember(Sokka)",
        "Scarcastic(Sokka)",
        "Prodigy(Azula)",
        "Villan(Azula)",
        "Lethal(Azula)",
    }
    rules = {
        "TeamAvatarMember(x) ^ Scarcastic(x) => Funny(x)",
        "Villan(y) ^ Lethal(y) ^ Prodigy(y) => Scary(y)", # Prodigy(y) swap y will swap y in both name and argument
    }
    engine_1 = Fol_engine(facts=facts, rules=rules)
    print(engine_1.isTrue("Funny(Sokka) ^ Scary(Azula)"))

test()

Test 7.1
Conjunct of {'Scary(Azula)', 'Funny(Sokka)'}
Checking Scary(Azula)
Rule match 'Villan(y) ^ Lethal(y) ^ Prodigy(y) => Scary(y)' for 'Scary(Azula)'
Swaping {'y': 'Azula'}
Swapped condition 'Villan(Azula) ^ Lethal(Azula) ^ Prodigy(Azula)'
Conjunct of {'Villan(Azula)', 'Lethal(Azula)', 'Prodigy(Azula)'}
Checking Villan(Azula)
Fact Function match 'Villan'
p_arguments ['Azula'], f_arguments ['Azula']
Fact 'Villan(Azula)' |=> Villan(Azula)
Checking Lethal(Azula)
Fact Function match 'Lethal'
p_arguments ['Azula'], f_arguments ['Azula']
Fact 'Lethal(Azula)' |=> Lethal(Azula)
Checking Prodigy(Azula)
Fact Function match 'Prodigy'
p_arguments ['Azula'], f_arguments ['Azula']
Fact 'Prodigy(Azula)' |=> Prodigy(Azula)
Rule Villan(Azula) ^ Lethal(Azula) ^ Prodigy(Azula) => Scary(Azula)
Checking Funny(Sokka)
Rule match 'TeamAvatarMember(x) ^ Scarcastic(x) => Funny(x)' for 'Funny(Sokka)'
Swaping {'x': 'Sokka'}
Swapped condition 'TeamAvatarMember(Sokka) ^ Scarcastic(Sokka)'
Conjunct of {'TeamAva

## Tests 6 and below

In [87]:
# Test 6
print("\nTest 1")
engine_1 = Fol_engine(facts={"Smart(Sokka)", "Funny(Sokka)"})
engine_1.isTrue("Smart(Sokka) ^ Funny(Sokka)")

print("\nTest 2")
engine_1 = Fol_engine(facts={"Smart(x)", "Funny(Sokka)"})
engine_1.isTrue("Smart(Sokka) ^ Funny(Sokka)")

print("\nTest 3")
engine_1 = Fol_engine(facts={"Smart(x)", "Funny(Sokka)"})
engine_1.isTrue("~Smart(Sokka) ^ Funny(Sokka)")

print("\nTest 4")
engine_1 = Fol_engine(facts={"Smart(x)", "~Funny(Sokka)"})
engine_1.isTrue("Smart(Sokka) ^ Funny(Sokka)")

print("\nTest 5")
engine_1 = Fol_engine(facts={"Smart(x)", "Funny(Sokka)"})
print(engine_1.isTrue("Smart(Sokka) ^ Funny(Azula)"))

print("\nTest 6")
engine_1 = Fol_engine(facts={"Smart(x)", "Funny(Sokka)", "~Funny(Azula)"})
print(engine_1.isTrue("Smart(Sokka) ^ Funny(Azula)"))


Test 1
Conjunct of {'Smart(Sokka)', 'Funny(Sokka)'}
Checking Smart(Sokka)
Fact Function match 'Smart'
p_arguments ['Sokka'], f_arguments ['Sokka']
Fact 'Smart(Sokka)' |=> Smart(Sokka)
Checking Funny(Sokka)
Fact Function match 'Funny'
p_arguments ['Sokka'], f_arguments ['Sokka']
Fact 'Funny(Sokka)' |=> Funny(Sokka)

Test 2
Conjunct of {'Smart(Sokka)', 'Funny(Sokka)'}
Checking Smart(Sokka)
Fact Function match 'Smart'
p_arguments ['Sokka'], f_arguments ['x']
Fact 'Smart(x)' |=> Smart(Sokka)
Checking Funny(Sokka)
Fact Function match 'Funny'
p_arguments ['Sokka'], f_arguments ['Sokka']
Fact 'Funny(Sokka)' |=> Funny(Sokka)

Test 3
Conjunct of {'~Smart(Sokka)', 'Funny(Sokka)'}
Checking ~Smart(Sokka)
Fact Function match 'Smart'
p_arguments ['Sokka'], f_arguments ['x']
Fact 'Smart(x)' |=> not ~Smart(Sokka)

Test 4
Conjunct of {'Smart(Sokka)', 'Funny(Sokka)'}
Checking Smart(Sokka)
Fact Function match 'Smart'
p_arguments ['Sokka'], f_arguments ['x']
Fact 'Smart(x)' |=> Smart(Sokka)
Checking Funn

In [25]:
# Test 5

print("\nTest 1")
engine_1 = Fol_engine(facts={"Clever(Rahul)"}, rules={"Clever(x) => Smart(x)"})
print( engine_1.isTrue("Smart(Rahul)") )


print("\nTest 2")
engine_1 = Fol_engine(facts={"Clever(y)"}, rules={"Clever(x) => Smart(x)"})
print( engine_1.isTrue("Smart(Rahul)") )

print("\nTest 3")
engine_1 = Fol_engine(facts={"Clever(y)"}, rules={"Clever(x) => ~Silly(x)"})
print( engine_1.isTrue("Silly(Rahul)") )

print("\nTest 4")
engine_1 = Fol_engine(facts={"~Clever(Rahul)"}, rules={"Clever(x) => Smart(x)"})
print( engine_1.isTrue("Smart(Rahul)") )

print("\nTest 5")
engine_1 = Fol_engine(facts={"Clever(Rahul)"}, rules={"~Clever(x) => Smart(x)"})
print( engine_1.isTrue("Smart(Rahul)") )


Test 1
Rule match 'Clever(x) => Smart(x)' for 'Smart(Rahul)'
Swaping {'x': 'Rahul'}
Swapped condition 'Clever(Rahul)'
Fact Function match 'Clever'
p_arguments ['Rahul'], f_arguments ['Rahul']
Fact 'Clever(Rahul)' |=> Clever(Rahul)
Rule Clever(Rahul) => Smart(Rahul)
True

Test 2
Rule match 'Clever(x) => Smart(x)' for 'Smart(Rahul)'
Swaping {'x': 'Rahul'}
Swapped condition 'Clever(Rahul)'
Fact Function match 'Clever'
p_arguments ['Rahul'], f_arguments ['y']
Fact 'Clever(y)' |=> Clever(Rahul)
Rule Clever(Rahul) => Smart(Rahul)
True

Test 3
Rule match 'Clever(x) => ~Silly(x)' for 'Silly(Rahul)'
Swaping {'x': 'Rahul'}
Swapped condition 'Clever(Rahul)'
Fact Function match 'Clever'
p_arguments ['Rahul'], f_arguments ['y']
Fact 'Clever(y)' |=> Clever(Rahul)
Rule Clever(Rahul) => ~Silly(Rahul)
False

Test 4
Rule match 'Clever(x) => Smart(x)' for 'Smart(Rahul)'
Swaping {'x': 'Rahul'}
Swapped condition 'Clever(Rahul)'
Fact Function match 'Clever'
p_arguments ['Rahul'], f_arguments ['Rahul']
Fact

In [ ]:
# Test 4

print("\nTest 1")
engine_1 = Fol_engine(facts={"~Clever(Rahul)"})
print( engine_1.isTrue("Clever(Rahul)") )

print("\nTest 2")
engine_1 = Fol_engine(facts={"~Clever(Rahul)"})
print( engine_1.isTrue("~Clever(Rahul)") )

print("\nTest 3")
engine_1 = Fol_engine(facts={"~Clever(x)"})
print( engine_1.isTrue("Clever(Rahul)") )

print("\nTest 4")
engine_1 = Fol_engine(facts={"~Clever(x)"})
print( engine_1.isTrue("~Clever(Rahul)") )

print("\nTest 5")
engine_1 = Fol_engine(facts={"~Clever(Rahul)"})
print( engine_1.isTrue("Clever(x)") )

print("\nTest 6")
engine_1 = Fol_engine(facts={"~Clever(Rahul)"})
print( engine_1.isTrue("~Clever(x)") )


Test 1
Fact Function match 'Clever'
p_arguments ['Rahul'], f_arguments ['Rahul']
Fact '~Clever(Rahul)' |=> not Clever(Rahul)
False

Test 2
Fact Function match 'Clever'
p_arguments ['Rahul'], f_arguments ['Rahul']
Fact '~Clever(Rahul)' |=> ~Clever(Rahul)
True

Test 3
Fact Function match 'Clever'
p_arguments ['Rahul'], f_arguments ['x']
Fact '~Clever(x)' |=> not Clever(Rahul)
False

Test 4
Fact Function match 'Clever'
p_arguments ['Rahul'], f_arguments ['x']
Fact '~Clever(x)' |=> ~Clever(Rahul)
True

Test 5
Fact Function match 'Clever'
p_arguments ['x'], f_arguments ['Rahul']
None

Test 6
Fact Function match 'Clever'
p_arguments ['x'], f_arguments ['Rahul']
None


In [ ]:
 # Test 3
'''

'''

print('Test 1')
engine_1 = Fol_engine(facts={"Has(x, Pen)"})
engine_1.isTrue("Has(Rahul, Pen)")

print('Test 2')
engine_1 = Fol_engine(facts={"OwnedBy(Pen, x)"})
engine_1.isTrue("OwnedBy(Pen, Rahul)")

Test 1
Fact Function match 'Has'
p_arguments ['Rahul', 'Pen'], f_arguments ['x', 'Pen']
Fact 'Has(x, Pen)' |=> Has(Rahul, Pen)
Test 2
Fact Function match 'OwnedBy'
p_arguments ['Pen', 'Rahul'], f_arguments ['Pen', 'x']
Fact 'OwnedBy(Pen, x)' |=> OwnedBy(Pen, Rahul)


True

In [ ]:
 # Test 2
'''
Engine should unify "Smart(Rahul)" with "Smart(x)"
first recognize "Smart(x)" is FuncArg
then recognize argument x is ground term, so itmight be unified with something
try to unify:
    success: is this the result?
    fail: move on
'''

print('Test 1')
engine_1 = Fol_engine(facts={"Smart(x)"})  # for all x, Smart(x) is true
engine_1.isTrue("Smart(Rahul)")

print("\nTest 2")
engine_1 = Fol_engine(facts={"Clever(x)", "Smart(y)"})
engine_1.isTrue("Smart(Rahul)")

print("\nTest 3")
engine_1 = Fol_engine(facts={"Clever(x)"})
engine_1.isTrue("Smart(Rahul)") == None

Test 1
Fact Function match 'Smart'
p_arguments ['Rahul'], f_arguments ['x']
Fact 'Smart(x)' |=> Smart(Rahul)

Test 2
Fact Function match 'Smart'
p_arguments ['Rahul'], f_arguments ['y']
Fact 'Smart(y)' |=> Smart(Rahul)

Test 3


True

In [ ]:
# Test 1
engine_1 = Fol_engine(facts={"Smart(Rahul)"})
engine_1.isTrue("Smart(Rahul)")

Fact Function match 'Smart'
p_arguments ['Rahul'], f_arguments ['Rahul']
Fact 'Smart(Rahul)' |=> Smart(Rahul)


True